In [1]:
# Run our imports
import pandas as pd
import numpy as np
import yfinance as yf
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import re
import time
import pickle
from fake_useragent import UserAgent


# INPUTS

In [2]:
# select Tickers
tickers = ['TSLA','AAPL','MSFT']

# DATA GATHERING

In [3]:
def get_links(tickers):

    for ticker in tickers:
        options = Options()
        options.add_argument(f'user-agent={UserAgent().random}')
        driver = webdriver.Chrome('../chromedriver',options=options)

        driver.get(f'https://seekingalpha.com/symbol/{ticker}/news')
        while True:
            time.sleep(10)
            if driver.find_elements_by_class_name("_1-r1S"):
                break
        SCROLL_PAUSE_TIME = 5
        
        reloads = 24
        for _ in range(reloads):
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
            
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(3)
            
        links = driver.find_elements_by_class_name("_1-r1S")
        df = pd.DataFrame()

        df['date'] = [i.find_element_by_class_name("VbEwc").text for i in links]
        df['name'] = [i.find_element_by_tag_name("h3").text for i in links]
        df['href'] = [i.find_elements_by_tag_name('a')[0].get_property('href') for i in links]
        df['ticker'] = [ticker] * len(df)        
        with open(f'../pickle_jar/training/{ticker}_news_history', 'wb') as fp:
            pickle.dump(df, fp)
        driver.close()
    
    df = pd.DataFrame()
    for t in tickers:
        dbfile = open(f'../pickle_jar/training/{t}_news_history', 'rb')      
        file = pickle.load(dbfile) 
        dbfile.close()        
        df = df.append(file, ignore_index=True)
    
    return df
        

In [132]:
df = get_links(tickers)


/Users/danielgurzi/opt/anaconda3/envs/DSI/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [4]:
df = pd.DataFrame()
for t in tickers:
    dbfile = open(f'../pickle_jar/training/{t}_news_history', 'rb')      
    file = pickle.load(dbfile) 
    dbfile.close()        
    df = df.append(file, ignore_index=True)
df

,date,name,href,ticker
0,"Today, 10:30 AM",Buying frenzy on Tesla stays energized,https://seekingalpha.com/news/3590612-buying-f...,TSLA
1,"Today, 9:01 AM",Tesla sets sights on $2K as another bear bails,https://seekingalpha.com/news/3590547-tesla-se...,TSLA
2,"Yesterday, 5:38 AM","Tesla drops prices again, cuts Model Y by $3,000",https://seekingalpha.com/news/3590421-tesla-dr...,TSLA
3,"Sat, Jul. 11",Li Auto to trade in the U.S. as investor inter...,https://seekingalpha.com/news/3590415-li-auto-...,TSLA
4,"Fri, Jul. 10","Tesla crosses $1,500 mark for the first time",https://seekingalpha.com/news/3590330-tesla-cr...,TSLA
...,...,...,...,...
1415,"Tue, Oct. 02, 2018",Microsoft could reveal new Surface models today,https://seekingalpha.com/news/3394509-microsof...,MSFT
1416,"Mon, Oct. 01, 2018",Freelance platform Upwork files for IPO,https://seekingalpha.com/news/3394199-freelanc...,MSFT
1417,"Fri, Sep. 28, 2018","Microsoft, Volkswagen collab on in-car cloud p...",https://seekingalpha.com/news/3393754-microsof...,MSFT
1418,"Fri, Sep. 28, 2018",WSJ: Slack prepping IPO for early next year,https://seekingalpha.com/news/3393694-wsj-slac...,MSFT


### Now we have a list of URLS from the seeking alpha ticker page

In [5]:
# this function will take the article links dataframe and go get the articles themselves 
# and save the information to a new dataframe

def qurom(df):
    #start a list of the articles
    art_list = []
    # for loop to go through each of the links and grab the articles
    options = Options()
    options.add_argument(f'user-agent={UserAgent().random}')
    driver = webdriver.Chrome('../chromedriver')    
    for link in df['href']:
        driver.get(link)
        
        while True:
#             if driver.find_element_by_tag_name("h1"):
            try: 
                driver.find_element_by_id("a-cont")
                break
            except:
                pass
            try: 
                driver.find_element_by_class_name('premium-banner')
                break
            except:
                pass
#             try:
#                 if driver.find_element_by_class_name('premium-banner'):
#                     break
#             except:
#                 True
        try: 
            article = driver.find_element_by_id("a-cont")
        except:
            article = 'premium-banner'
        # The Article has comments, which for now I am going to take out. This code will separate the article into two sections with "Like this article" as the separator
        sep = 'Recommended for you:'
        try: 
            art_list.append(article.text.split(sep,1)[0])
        except:
            art_list.append('error')
        with open(f'../pickle_jar/training/all_news_history', 'wb') as fp:
            pickle.dump(art_list, fp)
    driver.close()
    df['article'] = art_list
    return df

In [6]:
df2 = qurom(df)


In [8]:
with open(f'../pickle_jar/training/all_news_history_bu', 'wb') as fp:
    pickle.dump(df2, fp)

In [7]:
df2

,date,name,href,ticker,article
0,"Today, 10:30 AM",Buying frenzy on Tesla stays energized,https://seekingalpha.com/news/3590612-buying-f...,TSLA,Tesla (NASDAQ:TSLA) is now up 11.82% and trade...
1,"Today, 9:01 AM",Tesla sets sights on $2K as another bear bails,https://seekingalpha.com/news/3590547-tesla-se...,TSLA,The pain is too much for Roth Capital's Craig ...
2,"Yesterday, 5:38 AM","Tesla drops prices again, cuts Model Y by $3,000",https://seekingalpha.com/news/3590421-tesla-dr...,TSLA,The starting price for the Model Y crossover i...
3,"Sat, Jul. 11",Li Auto to trade in the U.S. as investor inter...,https://seekingalpha.com/news/3590415-li-auto-...,TSLA,Chinese EV player Li Auto (LI) files for a $10...
4,"Fri, Jul. 10","Tesla crosses $1,500 mark for the first time",https://seekingalpha.com/news/3590330-tesla-cr...,TSLA,Tesla (NASDAQ:TSLA) shares soared another 8% i...
...,...,...,...,...,...
1415,"Tue, Oct. 02, 2018",Microsoft could reveal new Surface models today,https://seekingalpha.com/news/3394509-microsof...,MSFT,Microsoft (NASDAQ:MSFT) is having a launch eve...
1416,"Mon, Oct. 01, 2018",Freelance platform Upwork files for IPO,https://seekingalpha.com/news/3394199-freelanc...,MSFT,Freelancer marketplace Upwork (NASDAQ:UPWK) fi...
1417,"Fri, Sep. 28, 2018","Microsoft, Volkswagen collab on in-car cloud p...",https://seekingalpha.com/news/3393754-microsof...,MSFT,Volkswagen (OTCPK:VWAGY -1.8%) and Microsoft (...
1418,"Fri, Sep. 28, 2018",WSJ: Slack prepping IPO for early next year,https://seekingalpha.com/news/3393694-wsj-slac...,MSFT,Microsoft (NASDAQ:MSFT) Teams competitor Slack...


In [21]:
def finish_qurom(df):
    driver = webdriver.Chrome('./chromedriver')
    #start a list of the articles
    art_list = []
    # grap the comments while I'm here
    comments = []
    # for loop to go through each of the links and grab the articles
    for r,c in df.iterrows():
        if c[3] == 'error':
            sep = 'Recommended for you:'
            driver.get(c[2])
            driver.implicitly_wait(50)
            try:
                if driver.find_element_by_class_name('premium-banner'):
                    df['article'][r] = 'premium-banner'
            except:
                try: 
                    df['article'][r] = driver.find_element_by_id("a-cont").text.split(sep,1)[0]
                except:
                    article = 'error'
                try: 
                    df['comments'][r] = driver.find_element_by_id("a-cont").text.split(sep,1)[1]
                except:
                    comments.append('error')
    driver.close()        
    
    return df

In [ ]:
df2[df2['name'].str.contains("volatile market")]
# df2.drop(df2.index[67], inplace=True)
df2 = df2[df2.article != 'premium-banner']


In [23]:
if 'error' in df2['article'].unique():
    print(df2['article'].str.count('error').sum())
    finish_qurom(df2)
else:
    print('Done')


Done


In [44]:
# save new dataframe to a csv
df2.to_csv(f'./data/{ticker}_articles.csv', index = False)


In [ ]:
df3 = pd.read_csv(f'./data/{ticker}_articles.csv')
df3.head()